# WIP: strawberry jupyter server

a work in progress of a `graphql` schema for jupyter server.

 https://strawberry.rocks/docs/general/schema-basics

    # %pip install 'strawberry-graphql[debug-server]'

In [1]:
    %pip install fastapi strawberry uvicorn jupyter_server IPython
    from jupyter_server.services.contents.largefilemanager import LargeFileManager
    from pathlib import Path
    contents = LargeFileManager(root_dir=str(Path("~").expanduser()))

In [2]:
    import strawberry as S, typing as T, strawberry.cli, enum
    from pathlib import Path
    from datetime import datetime

In [3]:
    DICT =  S.scalar(
        T.NewType("DICT", object),
        description="a dictionary",
        serialize=lambda v: v,
        parse_value=lambda v: v,
    )

In [4]:
    @strawberry.enum
    class ContentType(enum.Enum):
        FILE = "file"
        NOTEBOOK = "notebook"
        DIRECTORY = "directory"

    @S.type
    class Base:
        name: str
        path: str
        last_modified: datetime
        created: datetime
        format: str
        mimetype: T.Optional[str]
        size: T.Optional[int]
        writable: bool
        type: ContentType
        
        
    Cells = T.Union[DICT]
        
    @S.type
    class NbFormat:
        cells: list[Cells]
        metadata: DICT
    
    @S.type
    class Notebook(Base):
        content: DICT
    
    @S.type
    class File(Base):
        content: str
    
    @S.type
    class Directory(Base):
        content: T.Annotated["Kinds", "__main__"]

    Kinds = T.Union[Notebook, File, Directory]
    
    @S.type
    class Query:        
        @S.field
        def search(self, dir: str = "", kind: T.Optional[ContentType] = None) -> Kinds:
            return Directory(**contents.get(dir, ))
        
    schema = strawberry.Schema(Query)

In [5]:
    from fastapi import FastAPI
    from strawberry.fastapi import GraphQLRouter

    graphql_app = GraphQLRouter(schema)

    app = FastAPI()
    app.include_router(graphql_app, prefix="/graphql")

In [11]:
    if (I := "__file__" not in locals()):
        from IPython.display import IFrame
        import uvicorn, asyncio
        __import__("nest_asyncio").apply()

        config = uvicorn.Config(app)
        if "server" in locals():
            asyncio.ensure_future(server.shutdown())
        server = uvicorn.Server(config=config)

        asyncio.ensure_future(server.serve())

INFO:     Shutting down
INFO:     Started server process [288580]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


In [8]:
    I and display(IFrame("http://127.0.0.1:8000/graphql", *"100% 600".split()))

NameError: name 'I' is not defined

    asyncio.ensure_future(server.shutdown())